## 1. Import packages

In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. Data loader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

BATCH_SIZE = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


## 3. Modeling

In [3]:
def trainable_params(model):
    total_params = 0
    for param in model.parameters():
        if param.requires_grad:
            total_params += param.numel()

    print(f"Trainable params: {total_params/1e6:.2f} M")

In [4]:
from torchvision.models import vgg16, VGG16_Weights
vgg16_model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

vgg16_cls_ckpt = vgg16_model.classifier.state_dict()

In [5]:
trainable_params(vgg16_model)

Trainable params: 138.36 M


In [6]:
class CLS_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.features = vgg16_model.features.eval()
        for param in self.features.parameters():
            param.requires_grad_(False)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 1000),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(1000, 10),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
model = CLS_model().to(device)

In [8]:
trainable_params(model)

Trainable params: 123.65 M


## 4. Training  

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


In [10]:
model.train()

start = time.time()
for epoch in range(10):  
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{10}], Average Loss: {running_loss / len(trainloader):.4f}, GPU used: {torch.cuda.memory_allocated(0)/1e9:.2f} G')


print('Finished Training')
print(f'Training time: {time.time() - start:.2f} s')

Epoch [1/10], Average Loss: 1.3470, GPU used: 2.06 G
Epoch [2/10], Average Loss: 1.1362, GPU used: 2.06 G
Epoch [3/10], Average Loss: 1.1051, GPU used: 2.06 G
Epoch [4/10], Average Loss: 1.0829, GPU used: 2.06 G
Epoch [5/10], Average Loss: 1.0814, GPU used: 2.06 G
Epoch [6/10], Average Loss: 1.0621, GPU used: 2.06 G
Epoch [7/10], Average Loss: 1.0474, GPU used: 2.06 G
Epoch [8/10], Average Loss: 1.0420, GPU used: 2.06 G
Epoch [9/10], Average Loss: 1.0216, GPU used: 2.06 G
Epoch [10/10], Average Loss: 1.0221, GPU used: 2.06 G
Finished Training
Training time: 204.78 s


## 5. Evaluate

In [11]:
correct = 0
total = 0
model.eval()    
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 65.97 %
